<a href="https://colab.research.google.com/github/Andreas-Lukito/image_denoiser/blob/main/image_denoiser%20colab_ver.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Image Denoiser

In this project, an Autoencoder-based deep learning model is developed to reconstruct and denoise kirmizi data, simulating noisy sensor imagery. All input images are preprocessed and resized to 100×100 pixels to fit the network architecture, and Gaussian noise is added to mimic corrupted input data.

Two autoencoder models are would be compared:
1. A baseline Auto Encoder model with an architecture as follow:

| Layer         | Type         | Output Shape   | Kernel Size | Activation |
| ------------- | ------------ | -------------- | ----------- | ---------- |
| Input Layer   | Input        | (100, 100, 3)  | -           | -          |
| Conv2D_0      | Conv2D       | (100, 100, 32) | 3×3         | ReLU       |
| MaxPool2D_0   | MaxPooling2D | (50, 50, 32)   | 2×2         | -          |
| Conv2D_1      | Conv2D       | (50, 50, 64)   | 3×3         | ReLU       |
| MaxPool2D_1   | MaxPooling2D | (25, 25, 64)   | 2×2         | -          |
| Conv2D_2      | Conv2D       | (25, 25, 64)   | 3×3         | ReLU       |
| UpSample2D_0  | UpSampling2D | (50, 50, 64)   | -           | -          |
| Conv2D_3      | Conv2D       | (50, 50, 32)   | 3×3         | ReLU       |
| UpSample2D_1  | UpSampling2D | (100, 100, 32) | -           | -          |
| Conv2D_4      | Conv2D       | (100, 100, 3)  | 3×3         | Sigmoid    |


The optimizer for the baseline Auto Encoder model is Adam with a loss function of MSE.

2. A proposed Auto Encoder model optimized using Optuna for hyperparameter tuning.

All images were resized to 100x100 to match model input requirements. Gaussian noise was added to simulate corrupted data, useful for training the autoencoder to denoise images.


## Import Libraries

In [ ]:
!pip install optuna
!pip install pynvml
!pip install torch
!pip install torchvision
!pip install matplotlib
!pip install seaborn
!pip install pandas
!pip install scikit-learn
!pip install tqdm
!pip install scikit-image

In [ ]:

# Common python libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import pynvml
import os
import glob
from tqdm import tqdm
import time
from IPython.display import clear_output
import joblib
import shutil
import random
import copy
import cv2
from PIL import Image

# Deep Learning Libraries
import torch
import torch.nn as nn
from torch import optim
from torchvision import transforms
from torchvision.transforms import v2
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder


## GPU Check

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

if torch.cuda.is_available():
    print("Pytorch is using the GPU")

    pynvml.nvmlInit()

    num_gpus = pynvml.nvmlDeviceGetCount()

    for i in range(num_gpus):
        handle = pynvml.nvmlDeviceGetHandleByIndex(i)
        gpu_name = pynvml.nvmlDeviceGetName(handle)
        print("GPU Name:", gpu_name)

    pynvml.nvmlShutdown()
else:
    print("Pytorch is not using the GPU")

In [ ]:
print("Num GPUs Available: ", torch.cuda.device_count())

## Importing the Data

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

project_path = "/content/drive/MyDrive/project_image_denoiser"

In [ ]:
def img_to_df(img_path: str):
    """
    Convert image paths to DataFrame. for easier data preprocessing.

    Args:
        img_path (str): Path to the folder containing the image.

    Returns:
        pd.DataFrame: DataFrame containing image path, label, width, height, and channel_info.
    """
    #checks
    if not isinstance(img_path, str):
        raise TypeError("img_path must be a string.")
    if not os.path.exists(img_path):
        raise FileNotFoundError(f"The path {img_path} does not exist.")
    if not os.path.isdir(img_path):
        raise NotADirectoryError(f"The path {img_path} is not a directory.")

    # Get the list of all image files in the directory
    glob_path = os.path.join(img_path, "**", "*.jpg")
    print(f"glob_path: {glob_path}")

    image_path = glob.glob(glob_path, recursive=True) #recursive=True is to allow for subdirectories to be searched
    print(f"image_path: {image_path}")

    # Get the width, height, and channel information of the images
    width = []
    height = []
    channel_info = []
    avg_red = []
    avg_green = []
    avg_blue = []

    for path in tqdm(image_path, desc="Processing images", unit="image"):
        img = cv2.imread(path)
        if img is None:
            print(f"Skipping corrupted image: {path}")
            continue

        h, w, c = img.shape
        average = img.mean(axis=0).mean(axis=0)

        # Append to the list
        height.append(h)
        width.append(w)
        channel_info.append(c)
        avg_blue.append(average[0])
        avg_green.append(average[1])
        avg_red.append(average[2])


    #combine into one dataframe
    df = pd.DataFrame({
        "image_path": image_path,
        "width": width,
        "height": height,
        "channel_info": channel_info,
        "img_avg_red": avg_red,
        "img_avg_green": avg_green,
        "img_avg_blue": avg_blue
    })

    return df

### Caching Function to Load the Data Faster

In [ ]:
def save_cache(data, filename):
    """This is a function to save the data to a file using joblib

    Args:
        data (_type_): data to be saved
        filename (_type_): the name of the file to save the data to
    """
    with open(filename, 'wb') as f:
        joblib.dump(data, f)
    print(f"Data saved to {filename}")

def load_cache(filename):
    """This is a function to load the data to a file using joblib

    Args:
        data (_type_): data to be saved
        filename (_type_): the name of the file to load the data to
    """
    with open(filename, 'rb') as f:
        return joblib.load(f)
    print(f"Data loaded from {filename}")

In [ ]:
data_path = os.path.join(project_path, "dataset/")
write_path = "./Cache/"

if not os.path.exists(write_path):
            os.makedirs(write_path)

csv_path = os.path.join(write_path, "path_dataset.csv")

# We will cache the data so that it will load faster
if os.path.exists(csv_path):
    print("Loading cached dataset...")
    img_df = pd.read_csv(csv_path)
    print("Cached dataset loaded")
else:
    print("Creating and caching dataset...")
    img_df = img_to_df(data_path)
    img_df.to_csv(csv_path, index=False)

In [ ]:
img_df.head()

In [ ]:
#setting the colors generator
def fill_color_generator():
    """This generates a color

    Returns:
        color: An R,G,B value with a range of 0 to 1
    """
    r = random.randint(0, 255)
    g = random.randint(0, 255)
    b = random.randint(0, 255)
    return (r/255, g/255, b/255)

In [ ]:
#function for plotting numerical data distribution
def numeric_dist_plot(data: pd.DataFrame):
    """This function creates a plot of the distribution of the numerical data.

    Args:
        data (pd.DataFrame): Numeric pandas dataframe

    Raises:
        TypeError: The following columns are not numeric: {non_numeric_cols}
        This is due to some of the columns are not numeric.

    Returns:
        Displays a plot of
    """
    # Checks
    ## Check if all columns are numeric
    non_numeric_cols = [col for col in data.columns if not pd.api.types.is_numeric_dtype(data[col])]
    if non_numeric_cols:
        raise TypeError(f"The following columns are not numeric: {non_numeric_cols}")

    # Plotting the numerical data
    #titles for plots/figures
    fig_titles = []

    for colName in data.columns:
        fig_titles.append(f"Boxplot Of {colName}")
        fig_titles.append(f"Histogram Of {colName}")

    fill_color_dict = {}
    for colName in data.columns:
        fill_color_dict[colName] = fill_color_generator()

    # make subplot for each column name
    num_rows = len(data.columns)
    fig, axes = plt.subplots(nrows=num_rows, ncols=2, figsize=(12, num_rows * 3))

    # Flatten axes for easier indexing when there's more than 1 row
    axes = axes if num_rows > 1 else [axes]

    for i, column in enumerate(data.columns):
        color = fill_color_dict[column]

        # Boxplot
        axes[i][0].boxplot(data[column].dropna(), vert=False, patch_artist=True,
                        boxprops=dict(facecolor=color, color=color),
                        medianprops=dict(color="black"))
        axes[i][0].set_title(f"Boxplot of {column}")
        axes[i][0].set_xlabel(column)

        # Histogram
        axes[i][1].hist(data[column].dropna(), bins=20, color=color, alpha=0.7, edgecolor='black')
        axes[i][1].set_title(f"Histogram of {column}")
        axes[i][1].set_xlabel(column)

    # Overall layout
    fig.suptitle("Boxplot and Distribution Visualization for Each Numeric Column", fontsize=16)
    fig.tight_layout(rect=[0, 0, 1, 0.97])  # Adjust layout to fit title
    return plt

In [ ]:
numeric_data = img_df.select_dtypes(include=[np.number])
numeric_data.columns

In [ ]:
numeric_dist_plot(numeric_data)

## Preprocesing the images

In [ ]:
def create_datasets(df: pd.DataFrame, test_size: float = 0.1, val_size: float = 0.1):
    """
    Create train, validation, and test datasets.

    Args:
        df (pd.DataFrame): DataFrame containing image paths and labels.
        test_size (float): Proportion of the dataset to include in the test split.
        val_size (float): Proportion of the dataset to include in the validation split.
    """
    # Splitting the data into train and temp (which will be further split into validation and test)
    train_df, temp_df = train_test_split(df, test_size=(test_size + val_size), random_state=42)

    # Splitting train into validation and test sets
    val_df, test_df = train_test_split(temp_df, test_size=test_size / (test_size + val_size), random_state=42)

    # Make empty directories for the data
    data_path = "./input_data/"

    ## path to destination folders
    train_folder = os.path.join(data_path, 'train')
    val_folder = os.path.join(data_path, 'validation')
    test_folder = os.path.join(data_path, 'test')

    if os.path.exists(train_folder) and os.path.exists(val_folder) and os.path.exists(test_folder):
        print("Cached dataset directories found, skipping copying.")
        # Optionally: load the split info from saved CSVs
        train_df = pd.read_csv(os.path.join(data_path, "train_split.csv"))
        val_df = pd.read_csv(os.path.join(data_path, "val_split.csv"))
        test_df = pd.read_csv(os.path.join(data_path, "test_split.csv"))
        return train_df, val_df, test_df

    ## Create the directories
    for folder_path in [train_folder, val_folder, test_folder]:
        if not os.path.exists(folder_path):
            os.makedirs(folder_path)

    # Copy Images
    ## Train Images
    for img_path in tqdm(train_df['image_path'], desc="Copying Train Images"): #tqdm is used to show the progress bar
        filename = os.path.basename(img_path)
        os.makedirs(train_folder, exist_ok=True)
        shutil.copy(img_path, os.path.join(train_folder, filename))

    ## Test Images
    for img_path in tqdm(test_df['image_path'], desc="Copying Test Images"): #tqdm is used to show the progress bar
        filename = os.path.basename(img_path)
        os.makedirs(test_folder, exist_ok=True)
        shutil.copy(img_path, os.path.join(test_folder, filename))

    ## Validation Images
    for img_path in tqdm(val_df['image_path'], desc="Copying Validation Images"): #tqdm is used to show the progress bar
        filename = os.path.basename(img_path)
        os.makedirs(val_folder, exist_ok=True)
        shutil.copy(img_path, os.path.join(val_folder, filename))

    # Save CSVs for caching next time
    train_df.to_csv(os.path.join(data_path, "train_split.csv"), index=False)
    val_df.to_csv(os.path.join(data_path, "val_split.csv"), index=False)
    test_df.to_csv(os.path.join(data_path, "test_split.csv"), index=False)

    return train_df, val_df, test_df

In [ ]:
train_df, val_df, test_df = create_datasets(img_df)

### Add noise to the image

In [ ]:
class NoisyDataset(Dataset):
    def __init__(self, img_path, transform=None):
        self.img_path = img_path

        self.transform = v2.Compose([
            v2.Resize((100,100)),
            v2.ToImage(), # Convert to tensor
            v2.ToDtype(torch.float32, scale=True), #Scale to [0, 1]
            v2.GaussianNoise(mean=0.0, sigma=0.1, clip=True) # Add Gaussian noise
        ])

        self.clean_transform = v2.Compose([
            v2.Resize((100, 100)),
            v2.ToImage(),
            v2.ToDtype(torch.float32, scale=True)
        ])

    def __len__(self): #tells PyTorch how many samples the dataset has. To know how many batches to create, Avoid indexing errors, randomly shuffle or split data
        return len(self.img_path)

    def __getitem__(self, idx):
        img = Image.open(self.img_path[idx]).convert("RGB")
        noise_img = self.transform(img)
        clean_img = self.clean_transform(img)
        return noise_img, clean_img

## Making the Model

### Data Loader

In [ ]:
train_dataset = NoisyDataset(train_df['image_path'].values, transform=None)
val_dataset = NoisyDataset(val_df['image_path'].values, transform=None)
test_dataset = NoisyDataset(test_df['image_path'].values, transform=None)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

### Baseline Model

In [ ]:
class ConvAutoEncoder(nn.Module):
    def __init__(self):
        super(ConvAutoEncoder, self).__init__()
        self.model = nn.Sequential(
                #Conv2D_0
                nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3, stride=1, padding=1),
                nn.ReLU(),

                #MaxPool2d_0
                nn.MaxPool2d(kernel_size=2, stride=2),

                #Conv2D_1
                nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=1, padding=1),
                nn.ReLU(),

                #MaxPool2d_1
                nn.MaxPool2d(kernel_size=2, stride=2),
                nn.ReLU(),

                #Conv2D_2
                nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, stride=1, padding=1),
                nn.ReLU(),

                #UpSample2D_0
                nn.Upsample(scale_factor=2, mode='nearest'),

                #Conv2D_3
                nn.Conv2d(in_channels=64, out_channels=32, kernel_size=3, stride=1, padding=1),
                nn.ReLU(),

                #UpSample2D_1
                nn.Upsample(scale_factor=2, mode='nearest'),

                #Conv2D_4
                nn.Conv2d(in_channels=32, out_channels=3, kernel_size=3, stride=1, padding=1),
                nn.Sigmoid()
        )

    def forward(self, x):
        return self.model(x)

baseline_model = ConvAutoEncoder().to(device)


#### Train the model

In [ ]:
baseline_model = baseline_model.to(device)

loss_function = nn.MSELoss()
optimizer = optim.Adam(baseline_model.parameters(), lr=0.001)

In [ ]:
# Training loop
num_epochs = 20
train_loss = []
val_loss = []

for epoch in tqdm(range(num_epochs)):
    clear_output(wait=True)
    baseline_model.train()
    running_train_loss = 0

    for noisy_imgs, clean_imgs in train_loader:
        noisy_imgs = noisy_imgs.to(device)
        clean_imgs = clean_imgs.to(device)

        optimizer.zero_grad()
        outputs = baseline_model(noisy_imgs)
        loss = loss_function(outputs, clean_imgs)
        loss.backward()
        optimizer.step()

        running_train_loss += loss.item()

    avg_train_loss = running_train_loss/len(train_loader)
    train_loss.append(avg_train_loss)

    running_val_loss = 0
    for noisy_imgs, clean_imgs in val_loader:
        noisy_imgs = noisy_imgs.to(device)
        clean_imgs = clean_imgs.to(device)

        optimizer.zero_grad()
        outputs = baseline_model(noisy_imgs)
        loss = loss_function(outputs, clean_imgs)
        loss.backward()
        optimizer.step()

        running_val_loss += loss.item()

    avg_val_loss = running_val_loss/len(val_loader)
    val_loss.append(avg_val_loss)

    tqdm.write(f"Epoch [{epoch+1}/{num_epochs}]\nTrain_Loss: {avg_train_loss:.4f}\nVal_Loss: {avg_val_loss:.4f}")


In [ ]:
metrics = pd.DataFrame({
    "Train_Loss": train_loss,
    "Val_Loss" : val_loss
})

# Plot each feature
for col in metrics.columns:
    plt.plot(metrics[col], label=col, linewidth=1.5)

plt.xlabel("Metrics after each epoch")
plt.ylabel("Metric output")
plt.title("Metrics after each epoch in training")
plt.legend(loc='upper left')
plt.grid(True)
plt.tight_layout()
plt.show()

#### Model Evaluation

In [ ]:
def evaluate_autoencoder(model, dataloader, device):
    model.eval()
    mse_total = 0
    ssim_total = 0
    n_samples = 0

    with torch.no_grad():
        for noisy_imgs, clean_imgs in dataloader:
            noisy_imgs = noisy_imgs.to(device)
            clean_imgs = clean_imgs.to(device)

            outputs = model(noisy_imgs)

            # Compute MSE per batch
            batch_mse = torch.nn.functional.mse_loss(outputs, clean_imgs, reduction='sum').item()
            mse_total += batch_mse

            # Compute SSIM per image
            for i in range(outputs.size(0)):
                output_img = outputs[i].cpu().numpy().transpose(1, 2, 0) #convert from (Channels, Height, Width) → (Height, Width, Channels)
                clean_img = clean_imgs[i].cpu().numpy().transpose(1, 2, 0)

                # clip so that the minimum and maximum values in the image are 0 and 1 respectiveley
                output_img = np.clip(output_img, 0, 1)
                clean_img = np.clip(clean_img, 0, 1)

                ssim_score = ssim(output_img, clean_img, channel_axis=2, win_size=7, data_range=1.0)
                ssim_total += ssim_score

            n_samples += outputs.size(0)

    avg_mse = mse_total / n_samples
    avg_ssim = ssim_total / n_samples

    return avg_mse, avg_ssim

In [ ]:
avg_mse, avg_ssim = evaluate_autoencoder(baseline_model, test_loader, device)
print(f"MSE: {avg_mse:0.4f}")
print(f"SSIM: {avg_ssim:0.4f}")

MSE: 22.51 — reasonably low, depending on pixel value range (especially if [0, 1] scaled).

SSIM: 0.95+ — very high similarity, meaning it's reconstructing the images really well visually.

In [ ]:
os.makedirs(os.path.join(project_path, "models"), exist_ok=True) # Make directory in google drive
baseline_model_path = os.path.join(project_path,"models/best_baseline_model.pt")
new_model = baseline_model.to(device)
new_model.eval()

# Evaluate new model
new_mse, new_ssim = evaluate_autoencoder(new_model, test_loader, device)

if os.path.exists(baseline_model_path):
    # Load previous model
    old_model = ConvAutoEncoder().to(device)
    old_model.load_state_dict(torch.load(baseline_model_path))
    old_model.eval()

    old_mse, old_ssim = evaluate_autoencoder(old_model, test_loader, device)

    print(f"Old_RMSE = {old_mse:.4f} | New_RMSE = {new_mse:.4f}")
    print(f"Old_SSIM = {old_ssim:.4f} | New_SSIM = {new_ssim:.4f}")

    if new_mse < old_mse:
        print("🔁 New model is better — overwriting saved model.")
        torch.save(new_model.state_dict(), baseline_model_path)
        baseline_model = new_model
    else:
        print("✅ Existing model is still better.")
        baseline_model = old_model
else:
    print("New Model Performance")
    print(f"New_RMSE = {new_mse:.4f}")
    print(f"New_SSIM = {new_ssim:.4f}")
    print("📥 No existing model — saving new model.")
    os.makedirs(os.path.dirname(baseline_model_path), exist_ok=True)
    torch.save(new_model.state_dict(), baseline_model_path)
    baseline_model = new_model

### Proposed model

#### Define Model

In [ ]:
def define_model(trial):
    n_filters1 = trial.suggest_categorical("n_filters1", [16, 32, 64])
    n_filters2 = trial.suggest_categorical("n_filters2", [32, 64, 128])
    n_filters3 = trial.suggest_categorical("n_filters3", [64, 128, 256])
    upscaling_mode = trial.suggest_categorical("upscaling_mode", ['nearest', 'bilinear', 'bicubic'])

    # Encoder
    auto_enc = [
        nn.Conv2d(3, n_filters1, 3, stride=1, padding=1),
        nn.ReLU(),
        nn.MaxPool2d(2),
        nn.Conv2d(n_filters1, n_filters2, 3, stride=1, padding=1),
        nn.ReLU(),
        nn.MaxPool2d(2),

        # Bottleneck
        nn.Conv2d(n_filters2, n_filters3, 3, stride=1, padding=1),
        nn.ReLU(),
        nn.Conv2d(n_filters3, n_filters3, 3, stride=1, padding=1),
        nn.ReLU(),

        # Decoder (mirrored)
        nn.Upsample(scale_factor=2, mode=upscaling_mode),
        nn.Conv2d(n_filters3, n_filters2, 3, stride=1, padding=1),
        nn.ReLU(),
        nn.Upsample(scale_factor=2, mode=upscaling_mode),
        nn.Conv2d(n_filters2, n_filters1, 3, stride=1, padding=1),
        nn.ReLU(),
        nn.Conv2d(n_filters1, 3, 3, stride=1, padding=1),
        nn.Sigmoid()
    ]

    return nn.Sequential(*auto_enc)


In [ ]:
def objective(trial, train_loader, val_loader, epochs, device):
    best_mse = float("inf")  # Initialize with a high value
    epochs_no_improve = 0
    patience = 3

    model = define_model(trial).to(device)

    # Optimizer Set-Up
    optimizer_name = trial.suggest_categorical("Optimizer", ["Adam", "Momentum", "AdamW"])
    lr = trial.suggest_float("Learning_rate", 1e-5, 1e-1, log=True)

    optimizer = {
        "Adam": optim.Adam(model.parameters(), lr=lr),
        "Momentum": optim.SGD(model.parameters(),
                              lr=lr,
                              momentum=trial.suggest_float("sgd_momentum", 1e-5, 1, log=True)),
        "AdamW": optim.AdamW(model.parameters(),
                             lr=lr,
                             weight_decay=trial.suggest_float("adamw_weight_decay", 1e-5, 1, log=True))
    }[optimizer_name]

    loss_function = nn.MSELoss()

    for epoch in range(epochs):
        model.train()
        total_loss = 0.0

        for noisy, clean in train_loader:
            noisy, clean = noisy.to(device), clean.to(device)
            optimizer.zero_grad()
            outputs = model(noisy)
            loss = loss_function(outputs, clean)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()

        # Validation
        model.eval()
        val_loss = 0.0
        with torch.no_grad():
            for inputs, targets in val_loader:
                inputs, targets = inputs.to(device), targets.to(device)
                preds = model(inputs)
                loss = loss_function(preds, targets)
                val_loss += loss.item()

        avg_val_loss = val_loss / len(val_loader)

        # Early Stopping Logic
        if avg_val_loss < best_mse:
            best_mse = avg_val_loss
            epochs_no_improve = 0
            best_model_state = copy.deepcopy(model.state_dict())

        else:
            epochs_no_improve += 1
            if epochs_no_improve >= patience:
                print(f"Early stopping at epoch {epoch}")
                break

    #Save best model state
    if 'best_model_state' in locals():
        trial.set_user_attr("best_model_state", best_model_state)

    # Return the metric to minimize
    return best_mse


#### This will be continued in colab

#### Initiate Grid Search

In [ ]:
# Epochs and trials
epochs = 10
n_trials = 100

# Create the sampler and study
sampler = optuna.samplers.TPESampler(n_startup_trials=20)
study = optuna.create_study(direction="minimize", sampler=sampler)

# Progress bar
progress_bar = tqdm(total=n_trials, desc="Trial", ncols=80)

# Run the trials
for _ in range(n_trials):
    time.sleep(3)
    clear_output(wait=True)
    study.optimize(lambda trial: objective(trial, train_loader, val_loader, epochs, device), n_trials=1)
    progress_bar.update(1)

# Summarize study results
pruned_trials = study.get_trials(deepcopy=False, states=[TrialState.PRUNED]) # get the pruned trials
complete_trials = study.get_trials(deepcopy=False, states=[TrialState.COMPLETE]) # get the succeeded trials

print("\nStudy statistics:")
print(f"  Finished trials: {len(study.trials)}")
print(f"  Pruned trials:   {len(pruned_trials)}")
print(f"  Complete trials: {len(complete_trials)}")

# Define model with best params
best_trial = study.best_trial
print("Best trial:")
print(f"  MSE: {best_trial.value:.4f}")
best_proposed_model = define_model(best_trial).to(device)

# Load best weights
best_proposed_model.load_state_dict(best_trial.user_attrs["best_model_state"])

print("Best hyperparameters:")
for key, value in best_trial.params.items():
    print(f"  {key}: {value}")

In [ ]:
proposed_model_path = os.path.join(project_path,"models/best_proposed_model.pt")
new_model = define_model(best_trial).to(device)
new_model.load_state_dict(best_trial.user_attrs["best_model_state"])
new_model.eval()

# Evaluate new model
new_mse, new_ssim = evaluate_autoencoder(new_model, test_loader, device)

if os.path.exists(proposed_model_path):
    # Load previous model
    old_model = ConvAutoEncoder().to(device)
    old_model.load_state_dict(torch.load(proposed_model_path))
    old_model.eval()

    old_mse, old_ssim = evaluate_autoencoder(old_model, test_loader, device)

    print(f"Old_RMSE = {old_mse:.4f} | New_RMSE = {new_mse:.4f}")
    print(f"Old_SSIM = {old_ssim:.4f} | New_SSIM = {new_ssim:.4f}")

    if new_mse < old_mse:
        print("🔁 New model is better — overwriting saved model.")
        torch.save(new_model.state_dict(), proposed_model_path)
        proposed_model = new_model
    else:
        print("✅ Existing model is still better.")
        proposed_model = old_model
else:
    print("New Model Performance")
    print(f"New_RMSE = {new_mse:.4f}")
    print(f"New_SSIM = {new_ssim:.4f}")
    print("📥 No existing model — saving new model.")
    os.makedirs(os.path.dirname(proposed_model_path), exist_ok=True)
    torch.save(new_model.state_dict(), proposed_model_path)
    proposed_model = new_model